In [1]:
import numpy as np
from random import randint
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import copy
from importlib import reload  # Not needed in Python 2
import logging
import heapq
import math

In [2]:
TEAM_SIZE = 5
POS_TOP = 0
POS_JUG = 1
POS_MID = 2
POS_ADC = 3
POS_SUP = 4

In [3]:
class IndividualManager:
    def __init__(self, players={}, n_team=2, pos_weights=[1]*5, team_size=5):
        self.players = players
        self.n_team = n_team
        self.pos_weights = pos_weights
        self.team_size = team_size


    def get_individual(self):
        is_found_individual = False
        while not is_found_individual:
            is_found_individual = True
            individual = [0 for _ in range(self.team_size*self.n_team)]
            selected_pos = []
            n_player_selected = 0
            # players = random.sample(self.players.items(),k=self.team_size*self.n_team)
            for player_id, pos_score in self.players.items():
                player_pos_weights = pos_score.copy()*self.n_team
                for pos in selected_pos:
                    player_pos_weights[pos] = 0
                if sum(player_pos_weights) == 0:
                    is_found_individual = False
                    break
                player_pos = random.choices(range(self.team_size*self.n_team), k=1, weights=player_pos_weights)[0]
                selected_pos.append(player_pos)
                individual[player_pos] = player_id
        return individual
    

    # @FRONTEND: not implement needed
    def printable_player_name(self, name):
        MAX_LEN=8
        len_name = len(name)
        i = 0
        print_name = ''
        while i < MAX_LEN:
            if i < len_name:
                print_name += name[i]
            else:
                print_name += ' '
            i += 1
        return print_name


    def scoring(self, individual):
        team_scores  = []
        for i in range(self.n_team):
            team_score = 0
            pos_counter = 0
            for j in range(i*self.team_size,(i+1)*self.team_size):
                player_id = individual[j]
                player_score = self.players[player_id][pos_counter]
                if player_score == 0:
                    team_score = 0
                    continue
                team_score += self.pos_weights[pos_counter]*player_score
                pos_counter += 1
            team_scores.append(team_score)

        balance_team_score = 0
        for i in team_scores:
            for j in team_scores:
                balance_team_score += abs(i-j)
        return balance_team_score, team_scores


    # @FRONTEND: not implement needed
    def print_individual(self, individual):
        balance_team_score, team_scores = self.scoring(individual)
        print('       +-' + ('-'*23+'+')*self.n_team)
        dump_str = '       | \t'
        for i in range(1,self.n_team+1):
            dump_str += f'TEAM {i}  \t| \t'
        print(dump_str)
        rep_str = '| TOP: | \t'
        for ti in range(self.n_team):
            rep_str += self.printable_player_name(individual[ti*self.team_size+POS_TOP]) + ' \t| \t'
        len_rep_str = len(rep_str)
        dump_str = '+-------'
        for i in range((24)*self.n_team):
            dump_str += '-'
        dump_str += '+'
        print(dump_str)
        print(rep_str)
        print(dump_str)

        rep_str = '| JUG: | \t'
        for ti in range(self.n_team):
            rep_str += self.printable_player_name(individual[ti*self.team_size+POS_JUG]) + ' \t| \t'
        print(rep_str)
        print(dump_str)

        rep_str = '| MID: | \t'
        for ti in range(self.n_team):
            rep_str += self.printable_player_name(individual[ti*self.team_size+POS_MID]) + ' \t| \t'
        print(rep_str)
        print(dump_str)

        rep_str = '| ADC: | \t'
        for ti in range(self.n_team):
            rep_str += self.printable_player_name(individual[ti*self.team_size+POS_ADC]) + ' \t| \t'
        print(rep_str)
        print(dump_str)

        rep_str = '| SUP: | \t'
        for ti in range(self.n_team):
            rep_str += self.printable_player_name(individual[ti*self.team_size+POS_SUP]) + ' \t| \t'
        print(rep_str)
        print(dump_str)
        
        for i, team_score in enumerate(team_scores):
            print(f'TOTAL estimated score TEAM {i+1}: {team_score}')
        
        print(f'BALANCE: {balance_team_score}')


    def fitness_function(self, individual):
        balance_team_score, team_scores = self.scoring(individual)
        return (balance_team_score+1) / (sum(team_scores)+1)

In [4]:
# @FRONTEND: not implement needed
class Mutator:
    def __init__(self, individual_manager, n_team=2, team_size=5):
        self.n_team = n_team
        self.team_size = team_size
        self.individual_manager = individual_manager


    def exchange_players_in_same_position(self, individual):
        position = random.choice(range(self.team_size))
        temp = individual[position]
        for i in range(n_team-1):
            individual[i*self.team_size+position] = individual[(i+1)*self.team_size+position] 
        individual[(self.n_team-1)*team_size+position] = temp
        return individual


    def mutate(self, individual):
        new_individual = individual.copy()
        mutation_functions = [
            self.exchange_players_in_same_position,
        ]
        selected_mutation = random.choice(mutation_functions)
        new_individual = selected_mutation(new_individual)
        # print(new_individual)
        if individual_manager.fitness_function(new_individual) < individual_manager.fitness_function(individual):
            return individual
        return new_individual

In [5]:
# @FRONTEND: not implement needed
class Crossover:
    def __init__(self, n_team=2, team_size=5):
        self.n_team = n_team
        self.team_size = team_size


    def order1(self, parents):
        mom, dad = parents
        size = self.n_team*self.team_size

        # Choose random start/end position for crossover
        alice, bob = [-1] * size, [-1] * size
        start, end = sorted([random.randrange(size) for _ in range(2)])

        # Replicate mum's sequence for alice, dad's sequence for bob
        alice_inherited = []
        bob_inherited = []
        for i in range(start, end + 1):
            alice[i] = mom[i]
            bob[i] = dad[i]
            alice_inherited.append(mom[i])
            bob_inherited.append(dad[i])

        # print(alice, bob)
        #Fill the remaining position with the other parents' entries
        current_dad_position, current_mom_position = 0, 0

        fixed_pos = list(range(start, end + 1))
        i = 0
        while i < size:
            if i in fixed_pos:
                i += 1
                continue

            test_alice = alice[i]
            if test_alice==-1: #to be filled
                dad_trait = dad[current_dad_position]
                while dad_trait in alice_inherited:
                    current_dad_position += 1
                    dad_trait = dad[current_dad_position]
                alice[i] = dad_trait
                alice_inherited.append(dad_trait)

            test_bob = bob[i]
            if test_bob==-1: #to be filled
                mom_trait = mom[current_mom_position]
                while mom_trait in bob_inherited:
                    current_mom_position += 1
                    mom_trait = mom[current_mom_position]
                bob[i] = mom_trait
                bob_inherited.append(mom_trait)

            i +=1
        
        return alice, bob

In [6]:
def update_player_scores_by_histories(players, histories):
    updated_players = {}
    for player, scores in players.items():
        updated_players[player] = scores.copy()
        
    for history in histories:
        winners, losers, duration = history
        delta = 0.1
        if duration > 25:
            delta = 0.01
        elif duration < 16:
            delta = 0.2
        
        pos_counter = 0
        for winner in winners:
            updated_players[winner][pos_counter] += delta
            pos_counter += 1
        
        pos_counter = 0
        for loser in losers:
            updated_players[loser][pos_counter] -= delta
            pos_counter += 1
            
    return updated_players

In [7]:
class GASolver:
    def __init__(self, individual_manager, crossover, mutator, generations=1000, population_size=100, mutation_rate=0.1):
        self.individual_manager = individual_manager
        self.crossover = crossover
        self.mutator = mutator
        self.generations = generations
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.population = self.generate_initial_population()
        self.fitness_scores = self.evaluate_population(self.population)
        self.best_score = min(self.fitness_scores)
        # best_individual_idx = np.argmin(self.fitness_scores)
        self.best_individual = np.argmin(self.fitness_scores)
        # self.tournament_size = min(population_size/generations, 10)

    def generate_initial_population(self):
        return [self.individual_manager.get_individual() for _ in tqdm(range(self.population_size))]

    def evaluate_population(self, population):
        return [self.individual_manager.fitness_function(
            individual) for individual in population]

    def select_tournament_parents(self):
        NUMBER_PARENTS = 2
        parents = []
        for i in range(NUMBER_PARENTS):
            tournament_population_idx = random.sample(
                range(self.population_size), k=100)
            # tournament_fitness = self.evaluate_population(tournament_population)
            parents.append(random.choices([self.population[p] for p in tournament_population_idx], k=1, weights=[self.fitness_scores[p] for p in tournament_population_idx])[0])
        return parents

    def mutate(self, offspring):
        return [individual if random.random() > self.mutation_rate else self.mutator.mutate(individual) for individual in offspring]

    def update_population(self, offspring):
        offspring_fitness_score = []
        for i, off in enumerate(offspring):
            fitness_score = self.individual_manager.fitness_function(off)
            offspring_fitness_score.append(fitness_score)
            if fitness_score < self.best_score:
                self.best_score = fitness_score
                self.best_individual = self.population_size + i
                # print(f'New elite: {self.best_score} {self.best_individual}')
        self.population.extend(offspring)
        self.fitness_scores.extend(offspring_fitness_score)
        self.population_size += 2

    def visualize_fitness_progress(self, fitness_progress):
        plt.plot(fitness_progress)
        plt.title('Fitness Progress over Generations')
        plt.xlabel('Generations')
        plt.ylabel('Best Fitness Score')
        plt.show()

    def solve(self):
        fitness_progress = []
        for i in tqdm(range(self.generations)):
            fitness_progress.append(self.best_score)
            selected_parents = self.select_tournament_parents()
            offspring = self.crossover.order1(selected_parents)
            offspring = self.mutate(offspring)
            self.update_population(offspring)

        logging.info(f'Best solution: {self.best_score} {self.best_individual}')

        return self.best_individual

In [8]:
def get_players(player_ids):
    players = {}
    for player_id in player_ids:
        players[player_id] = PLAYERS[player_id].copy()
    return players

In [9]:
# DATA, GLOBAL CONSTANTS
HISTORIES = [
    # [winners, losers, duration]
    [['kkphan','mrbtb','thwd','joys','chocker'], ['zord','autobot','khaitun','pdlong','teddiss'], 19.6],
    [['zord','chocker','mrbtb','joys','khaitun'], ['nguoixau','thwd','bangnq','pdlong','autobot'], 16.5],
]

PLAYERS = {
    'mrbtb':        [0.0, 9.5, 6.0, 0.0, 0.0],
    'bangnq':       [0.0, 0.0, 7.0, 6.0, 0.0],
    'thwd':         [0.0, 6.5,  10,  10, 6.0],
    'joys':         [0.0, 0.0, 0.0, 5.0, 0.0],
    'autobot':      [0.0, 6.8, 0.0, 0.0,  10],
    'nguoixau':     [6.5, 0.0, 0.0, 8.0, 7.0],
    'zord':         [ 10, 0.0, 0.0, 7.0, 8.0],
    'toannm':       [0.0, 0.0, 0.0, 0.0, 9.0],
    'pdlong':       [0.0, 0.0, 7.0, 7.5, 8.0],
    'vietduc':      [7.0, 0.0, 0.0, 0.0, 0.0],
    'viethang':     [0.0, 0.0, 0.0, 0.0, 5.0],
    'teddiss':      [0.0, 0.0, 0.0, 0.0, 6.5],
    'michealdo':    [7.0, 9.0, 7.0, 0.0, 0.0],
    'chocker':      [9.0, 8.5, 8.5, 9.0, 9.0],
    'khaitun':      [0.0, 0.0,  10, 0.0, 8.0],
    'huongzang':    [6.0, 0.0, 0.0, 0.0, 8.0],
    'kkphan':       [9.5, 0.0, 0.0, 0.0, 0.0],
}

In [10]:
# INPUT
player_ids = ['kkphan','mrbtb','thwd','joys','chocker', 'zord','autobot','khaitun','pdlong','teddiss']
n_team = 2
team_size = 5
pos_weights = [1.0, 1.05, 1.0, 1.0, 0.9]

In [11]:
individual_manager = IndividualManager(players=get_players(player_ids), n_team=n_team, team_size=team_size, pos_weights=pos_weights)
individual_manager.print_individual(individual_manager.get_individual())
crossover = Crossover(n_team=n_team, team_size=team_size)
mutator = Mutator(individual_manager=individual_manager, n_team=n_team, team_size=team_size)

       +------------------------+-----------------------+
       | 	TEAM 1  	| 	TEAM 2  	| 	
+-------------------------------------------------------+
| TOP: | 	kkphan   	| 	zord     	| 	
+-------------------------------------------------------+
| JUG: | 	thwd     	| 	mrbtb    	| 	
+-------------------------------------------------------+
| MID: | 	khaitun  	| 	pdlong   	| 	
+-------------------------------------------------------+
| ADC: | 	chocker  	| 	joys     	| 	
+-------------------------------------------------------+
| SUP: | 	autobot  	| 	teddiss  	| 	
+-------------------------------------------------------+
TOTAL estimated score TEAM 1: 44.325
TOTAL estimated score TEAM 2: 37.825
BALANCE: 13.0


In [12]:
ga_solver = GASolver(
    individual_manager=individual_manager, 
    crossover=crossover,
    mutator=mutator,
    generations=1, 
    population_size=10000, 
    mutation_rate=0.01)
best_solution = ga_solver.solve()

100% 10000/10000 [00:08<00:00, 1141.39it/s]
100% 1/1 [00:00<00:00, 2464.34it/s]


In [14]:
individual_manager.print_individual(ga_solver.population[best_solution])

       +------------------------+-----------------------+
       | 	TEAM 1  	| 	TEAM 2  	| 	
+-------------------------------------------------------+
| TOP: | 	chocker  	| 	kkphan   	| 	
+-------------------------------------------------------+
| JUG: | 	autobot  	| 	mrbtb    	| 	
+-------------------------------------------------------+
| MID: | 	pdlong   	| 	khaitun  	| 	
+-------------------------------------------------------+
| ADC: | 	thwd     	| 	joys     	| 	
+-------------------------------------------------------+
| SUP: | 	zord     	| 	teddiss  	| 	
+-------------------------------------------------------+
TOTAL estimated score TEAM 1: 40.34
TOTAL estimated score TEAM 2: 40.325
BALANCE: 0.030000000000001137


In [16]:
elites_idx = np.argpartition(ga_solver.fitness_scores, 5)[:5]
i = 0
for idx in elites_idx:
    i += 1
    print(f'\n>>> Possible Solution {i} <<<')
    individual_manager.print_individual(ga_solver.population[idx])


>>> Possible Solution 1 <<<
       +------------------------+-----------------------+
       | 	TEAM 1  	| 	TEAM 2  	| 	
+-------------------------------------------------------+
| TOP: | 	chocker  	| 	kkphan   	| 	
+-------------------------------------------------------+
| JUG: | 	autobot  	| 	mrbtb    	| 	
+-------------------------------------------------------+
| MID: | 	pdlong   	| 	khaitun  	| 	
+-------------------------------------------------------+
| ADC: | 	thwd     	| 	joys     	| 	
+-------------------------------------------------------+
| SUP: | 	zord     	| 	teddiss  	| 	
+-------------------------------------------------------+
TOTAL estimated score TEAM 1: 40.34
TOTAL estimated score TEAM 2: 40.325
BALANCE: 0.030000000000001137

>>> Possible Solution 2 <<<
       +------------------------+-----------------------+
       | 	TEAM 1  	| 	TEAM 2  	| 	
+-------------------------------------------------------+
| TOP: | 	kkphan   	| 	chocker  	| 	
+------------------------